In [ ]:
import opendatasets as od
# Download the dataset
od.download("https://www.kaggle.com/datasets/yasserhessein/the-kvasir-dataset")

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the paths for source, training, testing, and validation
source_path = "the-kvasir-dataset/kvasir-dataset-v2"
train_path = "data/training"
test_path = "data/testing"
validation_path = "data/validation"

# Define the split ratios
train_ratio = 0.7
test_ratio = 0.2
validation_ratio = 0.1

# Create the directories
for path in [train_path, test_path, validation_path]:
    os.makedirs(path, exist_ok=True)    
    
# Process each class
for class_name in os.listdir(source_path):
    class_dir = os.path.join(source_path, class_name)
    if os.path.isdir(class_dir):
        # List all images
        images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        # Split the dataset
        train_val, test = train_test_split(images, test_size=test_ratio, random_state=42)
        train, val = train_test_split(train_val, test_size=validation_ratio/(train_ratio+validation_ratio), random_state=42)
        
        # Define a function to copy files
        def copy_files(filenames, dest_dir):
            os.makedirs(dest_dir, exist_ok=True)
            for f in filenames:
                shutil.copy(f, dest_dir)
                
        # Copy the files
        copy_files(train, os.path.join(train_path, class_name))
        copy_files(test, os.path.join(test_path, class_name))
        copy_files(val, os.path.join(validation_path, class_name))

# Delete the downloaded dataset
shutil.rmtree("the-kvasir-dataset")

In [4]:
import os

# Get the length of the training, testing, and validation datasets
train_path = "data/training/normal-z-line"
test_path = "data/testing/normal-z-line"
validation_path = "data/validation/normal-z-line"

print("Training: ", len(os.listdir(train_path)))
print("Testing: ", len(os.listdir(test_path)))
print("Validation: ", len(os.listdir(validation_path)))

Training:  699
Testing:  200
Validation:  101


In [5]:
import torch

# Import PyTorch and setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.__version__)
print(device)

2.4.0
cuda


In [6]:
%%writefile modular/data_setup.py
"""
Defines the functionality for creating PyTorch DataLoaders for the multi-class classification dataset.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str, 
                       test_dir: str, 
                       train_transform: transforms.Compose,
                       test_transform: transforms.Compose,
                       batch_size: int, 
                       num_workers: int=NUM_WORKERS):
    """Takes in a training and testing directory path and turns them into PyTorch DataLoaders.

    Args:
        train_dir (str): Path to training directory.
        test_dir (str): Path to testing directory.
        train_transform (transforms.Compose): Torchvision transforms to apply to the training dataset.
        test_transform (transforms.Compose): Torchvision transforms to apply to the testing dataset.
        batch_size (int): Number of samples per batch in each DataLoader.
        num_workers (int): Number of workers per DataLoader. Currently set to os.cpu_count().

    Returns:
        Tuple: Returns a tuple of (train_loader, test_loader, class_names). Where class_names is a dict of the target classes.
    """
    # Use ImageFolder to create datasets
    train_data = datasets.ImageFolder(root=train_dir,
                                      transform=train_transform)
    test_data = datasets.ImageFolder(root=test_dir,
                                     transform=test_transform)
    
    # Get the class names
    class_names = train_data.class_to_idx
    
    # Create DataLoaders
    train_loader = DataLoader(dataset=train_data, 
                              batch_size=batch_size, 
                              num_workers=num_workers, 
                              shuffle=True,
                              pin_memory=True)
    
    test_loader = DataLoader(dataset=test_data,
                             batch_size=batch_size,
                             num_workers=num_workers,
                             shuffle=False,
                             pin_memory=True)
    
    return train_loader, test_loader, class_names

Overwriting modular/data_setup.py


In [7]:
%%writefile modular/models/baseline_model.py
"""
Defines a PyTorch baseline model for multi-class classification.
"""
import torch
from torch import nn

class BaseLine(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        """Defines a simple feedforward neural network for multi-class classification.

        Args:
            input_shape (int): Number of input channels.
            hidden_units (int): Number of hidden units between layers.
            output_shape (int): Number of output units.
        """
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_shape),
            nn.ReLU()
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layer_stack(x)

Overwriting modular/models/baseline_model.py


In [8]:
%%writefile modular/engine.py
"""
Defines functions for training and testing PyTorch models.
"""
from typing import Tuple, List, Dict

import torch

from tqdm.auto import tqdm

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Turns the model into training mode and then runs through all the required training steps.

    Args:
        model (torch.nn.Module): A PyTorch model to be trained.
        dataloader (torch.utils.data.DataLoader): A DataLoader object for the training data.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to update the model weights.
        device (torch.device): A target device to send the data and model to.

    Returns:
        Tuple[float, float]: A tuple of the average loss and accuracy across all batches.
    """
    # Put the model in training mode
    model.train()
    
    #Setup the loss and accuracy
    train_loss, train_acc = 0, 0
    
    # Iterate over the data
    for batch, (X, y) in enumerate(dataloader):
        # Send the data to the device
        X, y = X.to(device), y.to(device)
        
        # Forward pass
        y_pred = model(X)
        
        # Get the predictions
        y_pred = model(X)
        
        # Calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
        
    # Adjust the metrics and get the avg loss and accuracy across all batches
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Turns the model into evaluation mode and then runs through all the required testing steps.

    Args:
        model (torch.nn.Module): A PyTorch model to be tested.
        dataloader (torch.utils.data.DataLoader): A DataLoader object for the testing data.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        device (torch.device): A target device to send the data and model to.

    Returns:
        Tuple[float, float]: A tuple of the average loss and accuracy across all batches.
    """
    # Put the model in evaluation mode
    model.eval()
    
    # Setup the loss and accuracy
    test_loss, test_acc = 0, 0
    
    # Turn on inference mode
    with torch.inference_mode():
        # Iterate over the data
        for batch, (X, y) in enumerate(dataloader):
            # Send the data to the device
            X, y = X.to(device), y.to(device)
        
            # Forward pass
            y_pred = model(X)
        
            # Calculate the loss
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
        
            # Calculate and accumulate accuracy metric across all batches
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            test_acc += (y_pred_class == y).sum().item()/len(y_pred)
        
    # Adjust the metrics and get the avg loss and accuracy across all batches
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_loader: torch.utils.data.DataLoader,
          test_loader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          scheduler: torch.optim.lr_scheduler,
          device: torch.device,
          epochs: int,
          patience: int = 5,
          min_delta: float = 0.001) -> Dict[str, List[float]]:
    """Passes a model through training and testing steps for a specified number of epochs.

    Args:
        model (torch.nn.Module): A PyTorch model to be trained and tested.
        train_loader (torch.utils.data.DataLoader): A DataLoader object for the training data.
        test_loader (torch.utils.data.DataLoader): A DataLoader object for the testing data.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to update the model weights.
        scheduler (torch.optim.lr_scheduler._LRScheduler): A PyTorch learning rate scheduler.
        device (torch.device): A target device to send the data and model to.
        epochs (int): The number of epochs to train the model for.
        patience (int): The number of epochs to wait before early stopping.
        min_delta (float): The minimum change in loss to be considered an improvement.

    Returns:
        Dict[str, List[float]]: A dictionary of lists containing the training and testing metrics.
    """
    # Setup a dict to store results
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}
    
    best_test_loss = float("inf")
    epochs_no_improve = 0
    best_model_state = None
    
    # Train the model
    for epoch in tqdm(range(epochs)):
        # Perform a training step
        train_loss, train_acc = train_step(model, train_loader, loss_fn, optimizer, device)
        
        # Perform a testing step
        test_loss, test_acc = test_step(model, test_loader, loss_fn, device)
        
        # Step the scheduler
        scheduler.step(test_loss)

        # Append the metrics to the dict
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # Print the metrics
        print(f"Epoch: {epoch+1}/{epochs} | Train Loss: {train_loss:.5f} | Train Acc: {train_acc:.5f} | Test Loss: {test_loss:.5f} | Test Acc: {test_acc:.5f}")
        
        # Print the current learning rate
        current_lr = optimizer.param_groups[0]["lr"]
        print(f"Current Learning Rate: {current_lr}")

        # Check for improvement
        if test_loss < best_test_loss - min_delta:
            best_test_loss = test_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1
            
        # Check for early stopping
        if epochs_no_improve == patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Load the best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print("Best model state loaded!")
        
    # Return the results at the end of the epochs
    return results

Overwriting modular/engine.py


In [9]:
%%writefile modular/utils.py
"""
Defines functions that contain various utility functions for PyTorch model training and saving.
"""
from pathlib import Path

import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str) -> None:
    """Save a PyTorch model to a specified directory.

    Args:
        model (torch.nn.Module): A PyTorch model to be saved.
        target_dir (str): The directory path to save the model.
        model_name (str): The name of the model file.
    """
    # Create the target directory
    Path(target_dir).mkdir(parents=True, exist_ok=True)
    
    # Create model save path
    assert model_name.endswith(".pt") or model_name.endswith(".pth"), "Model name must end with .pt or .pth"
    model_save_path = Path(target_dir) / model_name
    
    # Save the model
    print(f"Saving model to: {model_save_path}")
    torch.save(model.state_dict(), model_save_path)

Overwriting modular/utils.py


In [10]:
%%writefile modular/train.py
"""
Defines the training script for the PyTorch model.
"""
import os

import argparse

import inspect

import torch
from torchvision import transforms
import torchvision.models as models

import importlib.util

import sys
# Adjust the path to include the modular directory and where the scripts are located
script_dir = os.path.dirname(os.path.abspath(__file__))
sys.path.append("modular")
sys.path.append("modular/models")

import data_setup, engine, utils

# Function to list available models
def list_models():
    models_dir = os.path.join(script_dir, "models")
    model_files = [f for f in os.listdir(models_dir) if f.endswith(".py")]
    model_files = ["models/" + f for f in model_files]
    return ", ".join(model_files)

# Create ArgumentParser object
parser = argparse.ArgumentParser(description="Train a PyTorch multiclassification model on the colonoscopy dataset.")

# Add the arguments
parser.add_argument("--num_epochs", type=int, default=20, help="Number of epochs to train the model. Default is 20.")
parser.add_argument("--patience", type=int, default=5, help="Number of epochs to wait before early stopping. Default is 5.")
parser.add_argument("--min_delta", type=float, default=0.001, help="Minimum change in loss to be considered an improvement. Default is 0.001.")
parser.add_argument("--batch_size", type=int, default=32, help="Number of samples per batch. Default is 32.")
parser.add_argument("--learning_rate", type=float, default=0.001, help="Learning rate for the optimizer. Default is 0.001.")
parser.add_argument("--hidden_units", type=int, default=10, help="Number of hidden units in the model. Default is 10. Not needed for transfer learning models.")
parser.add_argument("--model_path", type=str, required=True, help=f"Path to the model file. Argument is required. Available models: {list_models()}")


# Parse the arguments
args = parser.parse_args()

# Setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
LEARNING_RATE = args.learning_rate
HIDDEN_UNITS = args.hidden_units
PATIENCE = args.patience
MIN_DELTA = args.min_delta

# Define the mapping of model names to their torchvision equivalents and default transformations
TRANSFER_LEARNING_MODELS = {
    "vgg19_model": models.VGG19_Weights.DEFAULT
}

# Import the specified model
model_script_path = os.path.join(script_dir, args.model_path)
spec = importlib.util.spec_from_file_location("model_module", model_script_path)
model_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(model_module)

def get_transforms(model_name):
    if model_name in TRANSFER_LEARNING_MODELS:
        weights = TRANSFER_LEARNING_MODELS[model_name]
        base_transform = weights.transforms()
        
        # Add data augmentation for training
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            base_transform
        ])
        
        # Use only the base transform for testing
        test_transform = base_transform
        
    else:
        # Default transforms if the model is not in TRANSFER_LEARNING_MODELS
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        test_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    return train_transform, test_transform

# Get the transformation based on the model name
model_name = os.path.basename(args.model_path).replace(".py", "")
train_transform, test_transform = get_transforms(model_name)

model_class = None
for name, obj in inspect.getmembers(model_module):
    if inspect.isclass(obj):
        model_class = obj
        break

if model_class is None:
    raise ValueError(f"Model class not found in {model_script_path}")

# Setup the directories
train_dir = "data/training"
test_dir = "data/testing"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create the DataLoaders using data_setup.py
train_loader, test_loader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=train_transform,
    test_transform=test_transform,
    batch_size=BATCH_SIZE
)

# Create the model
if model_name in TRANSFER_LEARNING_MODELS:
    model = model_class(output_shape=len(class_names), device=device).to(device)
else:
    model = model_class(input_shape=3,
                        hidden_units=HIDDEN_UNITS,
                        output_shape=len(class_names)).to(device)

# Set the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
# TODO: Figure out optimal optimizer and scheduler to use and the parameters to pass
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

# Start training the model using engine.py
from timeit import default_timer as timer

start_timer = timer()

engine.train(model=model,
train_loader=train_loader, 
test_loader=test_loader, 
loss_fn=loss_fn, 
optimizer=optimizer, 
scheduler=scheduler, 
device=device, 
epochs=NUM_EPOCHS,
patience=PATIENCE,
min_delta=MIN_DELTA)

end_timer = timer()

print(f"Training took: {end_timer - start_timer} seconds")

# Prompt the user to save the model
save_prompt = input("Do you want to save the model? (yes/no): ").lower()
if save_prompt == "yes":
    model_name = input("Enter the model name (without extension): ")
    utils.save_model(model, "saved_models", model_name + ".pth")
else: 
    print("Okay model will not be saved.")

Overwriting modular/train.py
